<a href="https://colab.research.google.com/github/ccosmin97/IDC-Prostate_segmentation/blob/main/results_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Docs/refs for viz**

*   [Plotly](https://plotly.com/python/box-plots/)
*   [Creating and Updating Figures in Python](https://plotly.com/python/creating-and-updating-figures/)
*   [Legends in Python](https://plotly.com/python/legend/)

#**Auth**

In [ ]:
#colab 
from google.colab import auth
auth.authenticate_user()

## Setup GCP Project ID

In [ ]:
project_id = "idc-nlst-unet-seg" # #YOUR_PROJECT ID

#**Imports**

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objects as go
import os
import pandas as pd
from google.cloud import bigquery
import numpy as np
import datetime

#**Dropbox setup for import csv results**

### Download results experiments csv from dropbox 

In [ ]:
path_csv_root = os.path.join(os.getcwd(), "results_csv")
if not os.path.exists(path_csv_root):
  !mkdir -p $path_csv_root

dropbox_links = ["https://www.dropbox.com/s/dd25sae3iwdw4mj/prostatex_task024-promise_3dfullres_analytics_metrics.csv?dl=0",
                 "https://www.dropbox.com/s/lw1l63nil195gtf/qin-rep-repeat_task024-promise_T2_3d-fullres_analysis_metrics.csv?dl=0",
                 "https://www.dropbox.com/s/uz59kp1lgrv3ssz/qin-rep-repeat_task005-prostate-pz_T2_3d-fullres_analysis_metrics.csv?dl=0",
                 "https://www.dropbox.com/s/v66hrmkhugfvbxd/qin-rep-repeat_task005-prostate-pz_ADC_analysis_metrics.csv?dl=0"]

for link in dropbox_links:
  name_file = link.split('/')[-1].split('?')[0]
  output_path = os.path.join(path_csv_root, name_file)
  !wget -O $output_path $link

--2022-07-18 15:42:00--  https://www.dropbox.com/s/dd25sae3iwdw4mj/prostatex_task024-promise_3dfullres_analytics_metrics.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/dd25sae3iwdw4mj/prostatex_task024-promise_3dfullres_analytics_metrics.csv [following]
--2022-07-18 15:42:00--  https://www.dropbox.com/s/raw/dd25sae3iwdw4mj/prostatex_task024-promise_3dfullres_analytics_metrics.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1db65fa3d6cbefccd84702e72e.dl.dropboxusercontent.com/cd/0/inline/BpXRhfZiwc9N0G5SQpkfmv9m5bSqJNn8jy15gYTayy_PJtl6dWTC1ufwtxn5jZxhu6Ow47UMpuMX-VfpIRJn5aQ-EWdNfE2yeMbfo3Geb1132P5d7aJTXY1tzpR0PMxhnzbYV_h5I2j0N4G8c6Op1mJGsN40ynORiFDGxTqZyGMmWQ/file# [following]
--2022-07-18 15:42:01--  h

In [ ]:
##setup paths
#task024 -- promise
promise_results_qin_rep_t2_path = os.path.join(path_csv_root, "qin-rep-repeat_task024-promise_T2_3d-fullres_analysis_metrics.csv")
promise_results_prostatex_t2_path = os.path.join(path_csv_root, "prostatex_task024-promise_3dfullres_analytics_metrics.csv")
#task005 -- prostate
pz_prostate_results_qin_rep_t2_path = os.path.join(path_csv_root, "qin-rep-repeat_task005-prostate-pz_T2_3d-fullres_analysis_metrics.csv")
pz_prostate_results_qin_rep_adc_path = os.path.join(path_csv_root, "qin-rep-repeat_task005-prostate-pz_ADC_analysis_metrics.csv")

#**Task 024 -- Promise -- T2 serie input**

## Summary




*   **Pre-trained model info**
  *  **Name Task** : Task 24 Promise
  *  **Input** : one modality : T2
  *  **Type of model** : 3d_fullres 
*   **IDC Data**
  * ProstateX : t2 scans only
  * Qin-Rep-Repeatability : t2 and adc scans, 2 StudyUIDS per PatientIDs, different timestamp



## Import results data - task24 promise

In [ ]:
promise_qin_rep_df = pd.read_csv(promise_results_qin_rep_t2_path) 
promise_prostatex_df = pd.read_csv(promise_results_prostatex_t2_path) 

In [ ]:
promise_qin_rep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   StudyInstanceUID        30 non-null     object 
 1   PatientID               30 non-null     object 
 2   dice_score              30 non-null     float64
 3   center_mass_ref         30 non-null     object 
 4   center_mass_pred        30 non-null     object 
 5   hausdorff_dist          30 non-null     float64
 6    ground_truth_seg_path  30 non-null     object 
 7    pred_nnnet_path        30 non-null     object 
dtypes: float64(2), object(6)
memory usage: 2.0+ KB


In [ ]:
promise_prostatex_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   StudyInstanceUID        66 non-null     object 
 1   PatientID               66 non-null     object 
 2   dice_score              66 non-null     float64
 3   center_mass_ref         66 non-null     object 
 4   center_mass_pred        66 non-null     object 
 5   hausdorff_dist          66 non-null     float64
 6    ground_truth_seg_path  66 non-null     object 
 7    pred_nnnet_path        66 non-null     object 
dtypes: float64(2), object(6)
memory usage: 4.2+ KB


In [ ]:
#concat dfs
promise_prostatex_df["collection"] = ["prostatex" for i in range(len(promise_prostatex_df))]
promise_qin_rep_df["collection"] = ["qin-rep-repeatability" for i in range(len(promise_qin_rep_df))]
promise_idc_df = pd.concat([promise_prostatex_df, promise_qin_rep_df])

## Viz of results

### Dice Score

In [ ]:
# scatter plot using plotly
fig = px.box(promise_idc_df,
             x = "collection",
             y="dice_score",
             points="all",
             color="collection",
             hover_data=["PatientID", "hausdorff_dist", "StudyInstanceUID", " pred_nnnet_path", " ground_truth_seg_path"],
             template="plotly_white",
             title="Dice Scores for prostatex and qin-rep-repeat for nnunet preds based on pre-trained model task24-promise, labelid=whole prostate",
             notched=True)
  
# initializing showlegend to "False"
fig.update_layout(showlegend=False)
  
fig.show()

#**Task 005 -- Prostate(Multi-modal -- T2+ADC) -- Qin-Rep-Repeatability**



## Label : Peripheral Zone

### Import results data - task005 prostate

In [ ]:
pz_prostate_qin_rep_t2_df = pd.read_csv(pz_prostate_results_qin_rep_t2_path)
pz_prostate_qin_rep_adc_df = pd.read_csv(pz_prostate_results_qin_rep_adc_path)
# pz_prostate_results_qin_rep_adc_2d_df = pd.read_csv(pz_prostate_results_qin_rep_adc_2d_path)

In [ ]:
pz_prostate_qin_rep_t2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   StudyInstanceUID        30 non-null     object 
 1   PatientID               30 non-null     object 
 2   dice_score              30 non-null     float64
 3   center_mass_ref         30 non-null     object 
 4   center_mass_pred        30 non-null     object 
 5   hausdorff_dist          30 non-null     float64
 6    ground_truth_seg_path  30 non-null     object 
 7    pred_nnnet_path        30 non-null     object 
dtypes: float64(2), object(6)
memory usage: 2.0+ KB


In [ ]:
pz_prostate_qin_rep_adc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   StudyInstanceUID        30 non-null     object 
 1   PatientID               30 non-null     object 
 2   dice_score              30 non-null     float64
 3   center_mass_ref         30 non-null     object 
 4   center_mass_pred        30 non-null     object 
 5   hausdorff_dist          30 non-null     float64
 6    ground_truth_seg_path  30 non-null     object 
 7    pred_nnnet_path        30 non-null     object 
dtypes: float64(2), object(6)
memory usage: 2.0+ KB


In [ ]:
#concat dfs
pz_prostate_qin_rep_t2_df["type_serie"] = ["resampled_to_T2" for i in range(len(pz_prostate_qin_rep_t2_df))]
pz_prostate_qin_rep_adc_df["type_serie"] = ["resampled_to_ADC" for i in range(len(pz_prostate_qin_rep_adc_df))]
# pz_prostate_results_qin_rep_adc_2d_df["type_serie"] = ["resampled_to_ADC_2D" for i in range(len(pz_prostate_results_qin_rep_adc_2d_df))]
pz_prostate_idc_df = pd.concat([pz_prostate_qin_rep_t2_df, pz_prostate_qin_rep_adc_df])#, pz_prostate_results_qin_rep_adc_2d_df])

### Viz of results

#### Dice Score

In [ ]:
# scatter plot using plotly
fig = px.box(pz_prostate_idc_df,
             x = "type_serie",
             y="dice_score",
             points="all",
             color="type_serie",
             hover_data=["PatientID", "hausdorff_dist", "StudyInstanceUID", " pred_nnnet_path", " ground_truth_seg_path"],
             template="plotly_white",
             title="Dice Scores for qin-rep-repeat for nnunet preds based on pre-trained model task005-prostate, labelid=Peripheral Zone",
             notched=True)
  
# initializing showlegend to "False"
fig.update_layout(showlegend=False)
  
fig.show()

### Further analysis

In [ ]:
pz_prostate_idc_df[pz_prostate_idc_df["PatientID"] == "PCAMPMRI-00010"].head()

,StudyInstanceUID,PatientID,dice_score,center_mass_ref,center_mass_pred,hausdorff_dist,ground_truth_seg_path,pred_nnnet_path,type_serie
18,1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299...,PCAMPMRI-00010,0.303778,-29.6323 26.4219 26.6483,-38.9712 19.911 28.6667,21.738583,/content/qin_prostate_root_data/results/data_s...,/content/qin_prostate_root_data/results/data_s...,resampled_to_T2
19,1.3.6.1.4.1.14519.5.2.1.3671.4754.318227959935...,PCAMPMRI-00010,0.404431,-21.6324 27.429 -92.6414,-34.0966 25.7832 -87.7737,28.307106,/content/qin_prostate_root_data/results/data_s...,/content/qin_prostate_root_data/results/data_s...,resampled_to_T2
18,1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299...,PCAMPMRI-00010,0.153208,-26.1119 22.9556 28.9902,-28.7709 16.9832 27.3767,20.732069,/content/qin_prostate_root_data/results/data_s...,/content/qin_prostate_root_data/results/data_s...,resampled_to_ADC
19,1.3.6.1.4.1.14519.5.2.1.3671.4754.318227959935...,PCAMPMRI-00010,0.122076,-16.5146 31.7607 -95.3018,-17.059 23.1911 -85.1679,20.943554,/content/qin_prostate_root_data/results/data_s...,/content/qin_prostate_root_data/results/data_s...,resampled_to_ADC


In [ ]:
pz_prostate_idc_df.sort_values(by=['hausdorff_dist'], ascending=True).head(100)

,StudyInstanceUID,PatientID,dice_score,center_mass_ref,center_mass_pred,hausdorff_dist,ground_truth_seg_path,pred_nnnet_path,type_serie
2,1.3.6.1.4.1.14519.5.2.1.3671.4754.304001015100...,PCAMPMRI-00002,0.802715,0.308765 -13.3908 -106.854,1.83022 -14.018 -105.434,5.929271,/content/qin_prostate_root_data/results/data_s...,/content/qin_prostate_root_data/results/data_s...,resampled_to_T2
5,1.3.6.1.4.1.14519.5.2.1.3671.4754.263464522348...,PCAMPMRI-00003,0.752261,-0.639141 -34.9351 27.5168,-2.24821 -34.1977 27.9303,5.999999,/content/qin_prostate_root_data/results/data_s...,/content/qin_prostate_root_data/results/data_s...,resampled_to_T2
6,1.3.6.1.4.1.14519.5.2.1.3671.4754.204078048920...,PCAMPMRI-00004,0.713306,-10.3119 -12.0416 -7.99657,-9.7745 -13.1695 -7.98419,6.257421,/content/qin_prostate_root_data/results/data_s...,/content/qin_prostate_root_data/results/data_s...,resampled_to_ADC
4,1.3.6.1.4.1.14519.5.2.1.3671.4754.162322077241...,PCAMPMRI-00003,0.728985,7.84778 -34.8263 26.4652,6.06932 -33.751 22.7927,6.416839,/content/qin_prostate_root_data/results/data_s...,/content/qin_prostate_root_data/results/data_s...,resampled_to_T2
7,1.3.6.1.4.1.14519.5.2.1.3671.4754.394942238206...,PCAMPMRI-00004,0.762944,-16.0423 -6.1594 11.1472,-16.4799 -6.67411 10.1558,6.655881,/content/qin_prostate_root_data/results/data_s...,/content/qin_prostate_root_data/results/data_s...,resampled_to_T2
20,1.3.6.1.4.1.14519.5.2.1.3671.4754.117496148504...,PCAMPMRI-00011,0.792099,0.176692 26.6068 -55.6962,0.0779215 26.0989 -57.6922,6.999959,/content/qin_prostate_root_data/results/data_s...,/content/qin_prostate_root_data/results/data_s...,resampled_to_T2
26,1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650...,PCAMPMRI-00014,0.723095,15.7349 32.1164 -12.1858,15.4209 31.8166 -14.1749,7.698455,/content/qin_prostate_root_data/results/data_s...,/content/qin_prostate_root_data/results/data_s...,resampled_to_T2
1,1.3.6.1.4.1.14519.5.2.1.3671.4754.288848219213...,PCAMPMRI-00001,0.626547,-5.74874 -4.97922 21.0434,-6.50662 -4.87997 19.0567,7.792141,/content/qin_prostate_root_data/results/data_s...,/content/qin_prostate_root_data/results/data_s...,resampled_to_ADC
3,1.3.6.1.4.1.14519.5.2.1.3671.4754.568383070573...,PCAMPMRI-00002,0.701878,-13.2565 -14.9826 -31.8808,-14.9344 -16.7749 -30.7833,7.860897,/content/qin_prostate_root_data/results/data_s...,/content/qin_prostate_root_data/results/data_s...,resampled_to_ADC
24,1.3.6.1.4.1.14519.5.2.1.3671.4754.211895462561...,PCAMPMRI-00013,0.692402,-16.5449 29.7639 -144.448,-15.7733 31.2163 -145.73,8.153029,/content/qin_prostate_root_data/results/data_s...,/content/qin_prostate_root_data/results/data_s...,resampled_to_ADC


#**Qin-Rep-Repeatability -- Study of consistency of predictions across same PatientID at different timestamps**

In [ ]:
def calculate_time_index(row, df_input): #assumes only two studies per PatientID
  other_study_time = df_input[(df_input["PatientID"] == row["PatientID"]) 
    & (df_input["StudyInstanceUID"] != row["StudyInstanceUID"])].StudyDate.values[0]
  if row.StudyDate < other_study_time:
    return "T0"
  elif row.StudyDate > other_study_time:
    return "T1"
  else:
    return "T"

## Promise pre-trained model

In [ ]:
selection_query = f"""
SELECT
  DISTINCT(StudyInstanceUID),
  PatientID,
  StudyTime,
  StudyDate
FROM
  `bigquery-public-data.idc_current.dicom_all`
WHERE
  collection_id = 'qin_prostate_repeatability'""" 

In [ ]:
bq_client = bigquery.Client(project_id)
selection_result_qin_rep_time = bq_client.query(selection_query)
selection_result_qin_rep_time_df = selection_result_qin_rep_time.result().to_dataframe()

In [ ]:
promise_qin_rep_df = promise_qin_rep_df.merge(selection_result_qin_rep_time_df[['StudyInstanceUID', 'StudyTime', 'StudyDate']],on='StudyInstanceUID', how='inner')
promise_qin_rep_df['time_index'] = promise_qin_rep_df.apply(calculate_time_index, args = (promise_qin_rep_df, ), axis=1)
promise_qin_rep_df["PID"] = promise_qin_rep_df.apply(lambda x: "PID_"+x["PatientID"].split('-')[1]+"_"+x["time_index"], axis=1)
promise_qin_rep_df = promise_qin_rep_df.sort_values(by=['PID'], ascending=True)

In [ ]:
# scatter plot using plotly
fig = px.line(promise_qin_rep_df,
             x = "PID",
             y="dice_score",
            #  points="all",
             color="PatientID",
             hover_data=["PatientID", "time_index", "dice_score", "hausdorff_dist", "StudyInstanceUID", "StudyDate", "StudyTime"],
             template="plotly_white",
             title="Dice Scores for qin-rep-repeat for nnunet preds based on pre-trained model task024-promise, labelid=whole prostate")

            #  notched=True)
  
# initializing showlegend to "False"

fig.update_layout(showlegend=False)

fig.show()

## Prostate-task005 pre-trained model

In [ ]:
selection_query = f"""
SELECT
  DISTINCT(StudyInstanceUID),
  PatientID,
  StudyTime,
  StudyDate
FROM
  `bigquery-public-data.idc_current.dicom_all`
WHERE
  collection_id = 'qin_prostate_repeatability'""" 

In [ ]:
bq_client = bigquery.Client(project_id)
selection_result_qin_rep_time = bq_client.query(selection_query)
selection_result_qin_rep_time_df = selection_result_qin_rep_time.result().to_dataframe()

In [ ]:
pz_prostate_qin_rep_t2_df = pz_prostate_qin_rep_t2_df.merge(selection_result_qin_rep_time_df[['StudyInstanceUID', 'StudyTime', 'StudyDate']],on='StudyInstanceUID', how='inner')
pz_prostate_qin_rep_t2_df['time_index'] = pz_prostate_qin_rep_t2_df.apply(calculate_time_index, args = (pz_prostate_qin_rep_t2_df, ), axis=1)
pz_prostate_qin_rep_t2_df["PID"] = pz_prostate_qin_rep_t2_df.apply(lambda x: "PID_"+x["PatientID"].split('-')[1]+"_"+x["time_index"], axis=1)
pz_prostate_qin_rep_t2_df = pz_prostate_qin_rep_t2_df.sort_values(by=['PID'], ascending=True)

In [ ]:
# scatter plot using plotly
fig = px.line(pz_prostate_qin_rep_t2_df,
             x = "PID",
             y="dice_score",
            #  points="all",
             color="PatientID",
             hover_data=["PatientID", "time_index", "dice_score", "hausdorff_dist", "StudyInstanceUID", "StudyDate", "StudyTime"],
             template="plotly_white",
             title="Dice Scores for qin-rep-repeat for nnunet preds based on pre-trained model task005-prostate, labelid=peripheral zone")

            #  notched=True)
  
# initializing showlegend to "False"

fig.update_layout(showlegend=False)

fig.show()